<a href="https://colab.research.google.com/github/JonathonLeiding/fifa-score-analysis/blob/main/fifa_neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [46]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [47]:
class Model(nn.Module):
  def __init__(self, in_features=43, h1=128, h2=64, h3=32, out_features=2):
        super().__init__()

        self.fc1 = nn.Linear(in_features, h1)
        self.fc2 = nn.Linear(h1, h2)
        self.fc3 = nn.Linear(h2, h3)
        self.out = nn.Linear(h3, out_features)

  def forward(self, x):
      x = F.leaky_relu(self.fc1(x))
      x = F.leaky_relu(self.fc2(x))
      x = F.leaky_relu(self.fc3(x))
      return self.out(x)


In [48]:
url = 'https://raw.githubusercontent.com/JonathonLeiding/fifa-score-analysis/refs/heads/main/jup_notebooks/cleaned_prem_data.csv'
df = pd.read_csv(url)

In [49]:
df.head(2)

,player_id,fifa_version,overall,potential,age,height_cm,weight_kg,club_team_id,club_jersey_number,preferred_foot,...,body_type_Lean (170-),body_type_Lean (170-185),body_type_Lean (185+),body_type_Normal (170-),body_type_Normal (170-185),body_type_Normal (185+),body_type_Stocky (170-),body_type_Stocky (170-185),body_type_Stocky (185+),body_type_Unique
0,653,23,91.0,91.0,1.335393,-0.193612,-0.786092,6,17.0,Right,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,29,23,90.0,90.0,2.658522,0.656983,0.892630,7,7.0,Right,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [50]:
player_ids = torch.tensor(df['player_id'].values, dtype=torch.long)
club_ids = torch.tensor(df['club_team_id'].values, dtype=torch.long)
df = df.drop(['primary_position', 'player_id', 'club_team_id'], axis=1)
df['preferred_foot'] = df['preferred_foot'].map({'Right': 0, 'Left': 1})

In [53]:
X_train = df[df['fifa_version'] < 22].drop(['fifa_version', 'goals','assists'], axis=1)
X_train = torch.FloatTensor(X_train.values)

X_test = df[df['fifa_version'] >= 22].drop(['fifa_version', 'goals','assists'], axis=1)
X_test = torch.FloatTensor(X_test.values)


In [54]:
Y_train = df[df['fifa_version'] < 22]
Y_train = Y_train[['goals','assists']]
Y_train = torch.FloatTensor(Y_train.values)

Y_test = df[df['fifa_version'] >= 22]
Y_test = Y_test[['goals','assists']]
Y_test = torch.FloatTensor(Y_test.values)

In [55]:
# Pick a seed for randomization
torch.manual_seed(31)
# Create an instance of a Model
model = Model()